In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cvxpy
import time

import multiprocessing as mp
from tqdm import tqdm

In [2]:
import io_utils
import kernels
import algos
from importlib import reload
reload(io_utils)

<module 'io_utils' from '/home/victor/Data/Mines-ParisTech/M2/mva/S2/km/kernel-methods/io_utils.py'>

In [33]:
np.random.seed(42)

raw_train, raw_val, labels_train, labels_val, feats_train, feats_val = [], [], [], [], [], []

for i in range(3):
    r_tr, r_val, feats_tr, feats_v, lbl_tr, lbl_val = io_utils.train_val_single_dataset(i)
    raw_train.append(r_tr)
    raw_val.append(r_val)
    labels_train.append(lbl_tr)
    labels_val.append(lbl_val)
    feats_train.append(feats_tr)
    feats_val.append(feats_v)

In [18]:
pool = mp.Pool(6)
# Compute kernels
kernel_train, kernel_vals = [], []

kern_ss = kernels.SubstringKernel(7, 0.1)

for n in range(3):
    print(n)
    n_train, n_val = raw_train[n].shape[0], raw_val[n].shape[0]
    
    t = time.time()
    args = [(raw_train[n][i], raw_train[n][j]) for i in range(n_train) for j in range(i, n_train) ]
    result = pool.map(kern_ss.computeKernel, args)
    e = time.time()
    print(e-t)
    
    count = 0
    K = np.zeros((n_train, n_train))
    for i in range(n_train):
        for j in range(i, n_train):
            K[i,j] = result[count]
            count +=1

    for i in range(n_train):
        for j in range(0, i):
            K[i,j] = K[j,i]
  
    kernel_train.append(K.copy())
    args = [(raw_train[n][i], raw_val[n][j]) for i in range(n_train) for j in range(n_val) ]
    
    result = pool.map(kern_ss.computeKernel, args)
    intermediate = np.zeros((n_train, n_val))
    count = 0
    for i in tqdm(range(n_train)):
        for j in range(n_val):
            intermediate[i,j] = result[count]
            count +=1
    kernel_vals.append(intermediate.copy())



0


NameError: name 'raw_train' is not defined

Process ForkPoolWorker-5:
Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessin

In [36]:
for i in range(3):
    np.savetxt('kernels/substring_3_01_train_train_{}.txt'.format(i), kernel_train[i])
    np.savetxt('kernels/substring_3_01_train_val_{}.txt'.format(i), kernel_vals[i])
    np.savetxt('kernels/substring_3_01_labels_train_{}.txt'.format(i), labels_train[i])
    np.savetxt('kernels/substring_3_01_labels_val_{}.txt'.format(i), labels_val[i])
    with open('kernels/substring_3_01_raw_train_{}.txt'.format(i), 'w') as f:
        f.write("\n".join(raw_train[i]))
    
    with open('kernels/substring_3_01_raw_train_{}.txt'.format(i), 'w') as f:
        f.write("\n".join(raw_val[i]))

In [77]:
for i in range(3):
    np.savetxt('kernels/substring_3_01_feats_train_{}.txt'.format(i), feats_train[i])
    np.savetxt('kernels/substring_3_01_feats_val_{}.txt'.format(i), feats_val[i])


In [5]:
# Test
kernel_tests = []

for i in range(3):
    cur_test = io_utils.load_raw_test(i)
    
    args = [(raw_train[i][j], cur_test[k]) for j in range(raw_train[i].shape[0]) for k in range(cur_test.shape[0]) ]
    result = pool.map(computeKernel, args)

    kernel_test = np.zeros((raw_train[i].shape[0], cur_test.shape[0]))

    count = 0
    for k in range(raw_train[i].shape[0]):
        for l in range(cur_test.shape[0]):
            kernel_test[k,l] = result[count]
            count +=1
            
    kernel_tests.append(kernel_test.copy())

NameError: name 'computeKernel' is not defined

## Start here to load saved kernels

In [30]:
# Load from files
reload(io_utils)
kernel_train, labels_train, kernel_vals, labels_val, feats_train, feats_val, _ = io_utils.load_kernels("kernels/substring_3_01")

### Linear kernel

In [25]:
linear_kernel = kernels.LinearKernel()
K_linear_train, K_linear_val = [], []
for i in range(3):
    K_linear_train.append(linear_kernel.computeVectorizedKernel(feats_train[i], feats_train[i]))
    K_linear_val.append(linear_kernel.computeVectorizedKernel(feats_train[i], feats_val[i]))    

In [28]:
## Prediction 
lambd = 7e-1
lambd2 = 7e-6
TRIALS = 30

lambda_vals = np.logspace(-7, -3, TRIALS)

all_preds = []

krr = algos.KernelRidgeRegression()

weights = np.ones((3,2))

for i in range(3):
    krr.fit(kernel_train[i], labels_train[i], lambd)
    error_1 = krr.evaluatePerformance(kernel_vals[i], labels_val[i])
    predictions_1 = np.einsum('i, ij->j', krr.alpha, kernel_vals[i])

    krr.fit(K_linear_train[i], labels_train[i], lambd2)
    error_2 = krr.evaluatePerformance(K_linear_val[i], labels_val[i])
    w_1 = (1-error_2)/(1-error_1)
    predictions_2 = np.einsum('i, ij->j', krr.alpha, K_linear_val[i])

    global_error = 1 / (w_1 + 1) * predictions_1 + w_1 / (w_1 + 1) * predictions_2
    print(np.round(1-np.sum(np.sign(global_error) != np.sign(labels_val[i])) / labels_val[i].shape[0],3))
    print("----")

# Test
#     pred_test = krr.predict(kernel_tests[i])
#     all_preds.append(pred_test.copy())

    

Score : 0.605
[-2.43244832e-01 -2.08579400e-01  1.17951894e-01 -7.29764380e-03
 -3.17318298e-01 -1.92014371e-01 -1.08201944e-01 -9.74937831e-02
 -8.46448309e-02  5.55066073e-02 -3.46158804e-01 -2.05719887e-01
  2.89965864e-01  2.68665420e-01  4.24324376e-01 -1.15634030e-01
 -1.79287925e-02  3.36308571e-01  6.40214100e-02  2.00127842e-01
 -4.03757671e-01 -8.42313921e-02 -5.07216586e-01 -3.27855418e-01
 -5.31604721e-02 -1.00802490e-01  1.91197914e-01 -1.23844671e-01
 -3.51125440e-01  1.58846473e-01 -1.91132331e-02  1.55407527e-01
  3.19683769e-01 -2.09490338e-01 -1.42364901e-01  1.74738324e-01
  3.21066919e-01  2.42611195e-01 -1.01620285e-01  1.37794829e-01
 -1.84873878e-01  3.89505696e-01  4.46049570e-02 -2.23181033e-03
  1.78012669e-01 -4.94371546e-01 -1.66567709e-01 -2.21550237e-01
 -1.51553917e-01 -1.83721434e-01 -3.99678122e-02 -2.35580382e-01
  1.38677034e-01  6.35761430e-03  9.74875763e-02  1.44794365e-01
 -1.55370610e-01  2.96231664e-01  1.68300776e-01 -8.14862428e-02
 -3.4007151

Score : 0.633
[ 4.81697123e-01 -3.89746921e-01  6.95339124e-02 -4.54635965e-03
  3.65639392e-02 -1.04238669e-01  1.84188736e-01 -2.99695134e-02
  4.48204911e-01  4.15862481e-01  2.59097570e-02 -1.51280931e-01
  2.43575393e-01  3.77528931e-02 -1.23653126e-01  7.83987337e-02
  1.08943116e-01  5.36447613e-03 -3.18297261e-01 -1.03336390e-01
 -1.50126252e-01 -1.16572011e-02 -1.66087808e-01  4.48857499e-01
  7.83317987e-02 -1.42646453e-01 -1.96868893e-01 -8.91739894e-02
  2.82700164e-01  6.39814540e-02  1.69415944e-01 -1.91377883e-01
 -1.70359801e-01 -2.22326344e-01  1.57821466e-01  1.53492267e-02
  2.51556282e-01  5.71544548e-03 -1.91496927e-01  6.00701516e-02
  1.92858337e-01 -1.46299290e-01  3.00607837e-01  3.36655694e-02
 -7.92042560e-02  3.98435304e-02 -7.80110739e-02  2.79154081e-01
 -2.17687558e-01  5.28071264e-02  7.80363748e-02  6.95986859e-02
  7.95292787e-02  4.19443727e-01  3.66576639e-02  2.57660792e-01
 -2.93944228e-01 -3.44668027e-01  1.69202634e-01  1.17843881e-02
  1.3123574

Score : 0.690
[ 0.02997861 -0.21215069  0.73587593 -0.4636156   0.95552717  0.22473067
  0.60044543  0.41425097 -0.08049346 -0.44167075 -0.3873828   0.05534875
 -0.42416565 -0.22330959  0.01666887  0.68787312  0.40930033 -0.57915728
  1.13057536 -0.20880191  0.22664009  0.16791701 -0.26018773 -0.54405944
  0.63796847  0.6912952  -0.43736285 -0.05233757 -0.32427869 -1.02725954
 -0.13597131  0.06818445 -0.02823971 -0.30876459 -0.88397058 -0.0192568
  0.15263882  0.90150037  0.31083223 -0.27527076  0.20501395 -0.46809064
  0.69311666  0.72505895 -0.72384229  0.23227603 -0.08862798  0.07815955
  0.40376262 -0.31997897 -0.3493656   0.28447638  0.45114994 -0.42566991
 -0.04746553 -0.57235762  0.64936667  0.08584375 -0.04247443 -0.59799343
 -0.626368    0.16006202  0.21896671  0.0270367   0.20750559 -0.10556774
  0.63205451  0.85487908 -0.03935321 -0.27236974  0.30077803 -0.17913269
  0.86657473 -0.261633   -0.11997548  0.10409753 -0.5098307   0.14096658
 -0.46073717 -0.22127608 -0.58131384  

In [42]:
TRIALS = 10

lambda_vals = np.logspace(-2, 3, TRIALS)

krr = algos.KernelRidgeRegression()
for i in range(3):
    errors = krr.searchBestParameters(kernel_train[i], train_labels[i], kernel_vals[i], val_labels[i], lambda_vals)

NameError: name 'train_labels' is not defined

In [21]:
# Load data
k = 0
np.random.seed(42)
shuffled_idx = np.arange(n)
np.random.shuffle(shuffled_idx)
VALIDATION_RATIO = 0.2
kernels_train, labels_train, kernels_val, labels_vals, feats_train, feats_val = io_utils.load_kernels("kernels/substring_3_01")
K_tr_tr_substring = kernels_train[k]
K_tr_tt_substring = kernels_val[k]
y_train, y_test = labels_train[k], labels_vals[k]


# Compute predictions
test_algo = algos.KernelRidgeRegression()
test_algo.fit(K_tr_tr_substring, y_train, 0.7)
test_algo.evaluatePerformance(K_tr_tr_substring, y_train)
test_algo.evaluatePerformance(K_tr_tt_substring, y_test)

Score : 0.607
Score : 0.605


0.395